In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import Concatenate, Lambda # batch discrimination
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Sequential, Model
from keras.optimizers import Adam

import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

import skimage.measure

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [3]:
def add_common_layers(y, set_alpha=0.3):
    #y = BatchNormalization(momentum=0.8)(y)
    y = BatchNormalization()(y)
    #y = Dropout(0.25)(y)
    y = LeakyReLU(alpha=set_alpha)(y)

    return y

In [4]:
class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 3
        self.latent_dim = 128

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))
        model_input = multiply([noise, label_embedding])
        
        

        i = Reshape((1, 1, 128), input_shape=(128,))(model_input)
        
        i = Conv2DTranspose(128, kernel_size=4, padding="valid")(i)
        i = add_common_layers(i)
        
        i = Conv2DTranspose(128, kernel_size=5, padding="valid")(i)
        i = add_common_layers(i)
        
        i = UpSampling2D(size=(2, 2))(i)
        i = Conv2D(64, kernel_size=3, strides=1, padding="same")(i)
        i = add_common_layers(i, set_alpha=0)
        
        i = UpSampling2D(size=(2, 2))(i)
        i = Conv2D(32, kernel_size=3, strides=1, padding="same")(i)
        i = add_common_layers(i, set_alpha=0)
        
        i = UpSampling2D(size=(2, 2))(i)
        i = Conv2D(16, kernel_size=3, strides=1, padding="same")(i)
        i = add_common_layers(i, set_alpha=0)
        
        i = Conv2D(3, kernel_size=3, strides=1, padding="same")(i)
        #i = LeakyReLU(alpha=0)(i)

        img = Activation("tanh")(i)
        
        
        model = Model([noise, label], img)
        model.summary()

        return model

    def build_discriminator(self):
        # batch_diversity = batch discrimination
        img = Input(shape=self.img_shape)

        i = Conv2D(16, kernel_size=3, strides=1, input_shape=(64,64,3), padding="same")(img)
        i = add_common_layers(i)
        batch_div = Lambda(lambda x:K.mean(K.abs(x[:] - K.mean(x,axis=0)),axis=-1,keepdims=True))(i)
        i = Concatenate()([i, batch_div])
        
        i = Conv2D(32, kernel_size=3, strides=2, padding="same")(i)
        i = add_common_layers(i)
        batch_div = Lambda(lambda x:K.mean(K.abs(x[:] - K.mean(x,axis=0)),axis=-1,keepdims=True))(i)
        i = Concatenate()([i, batch_div])
        
        i = Conv2D(64, kernel_size=3, strides=2, padding="same")(i)
        i = add_common_layers(i)
        batch_div = Lambda(lambda x:K.mean(K.abs(x[:] - K.mean(x,axis=0)),axis=-1,keepdims=True))(i)
        i = Concatenate()([i, batch_div])
        
        i = Conv2D(128, kernel_size=3, strides=2, padding="same")(i)
        #i = LeakyReLU(alpha=0.2)(i)
        #i = Dropout(0.25)(i)

        i = Flatten()(i)
        
        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(i)
        label = Dense(self.num_classes, activation="softmax")(i)
        
        model = Model(img, [validity, label])
        model.summary()  

    
        return model


    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        data = np.load("AWS3HAMNOAUG_256.npz")
        X_train = data["imageList"]
        y_train = data["targetList"]
        y_train = np.argmax(y_train, axis=1)

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Resize X_train from 256x256 to 64x64
        temp = np.empty((7133, 64, 64, 3))
        for i in range(X_train.shape[0]):
            temp_r = skimage.measure.block_reduce(X_train[i][:,:,0], (4,4), np.mean) # (4,4) = factor of reduction
            temp_g = skimage.measure.block_reduce(X_train[i][:,:,1], (4,4), np.mean)
            temp_b = skimage.measure.block_reduce(X_train[i][:,:,2], (4,4), np.mean)
            temp_rgb = np.stack([temp_r, temp_g, temp_b], axis=-1)

            temp[i] = temp_rgb    
        X_train = temp
        

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # The labels of the digits that the generator tries to create an
            # image representation of
            sampled_labels = np.random.randint(0, 3, (batch_size, 1))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, sampled_labels])

            # Image labels. 0-2 
            img_labels = y_train[idx]

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, [valid, img_labels])
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, sampled_labels])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.save_model(epoch)
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 10, 3
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c, figsize=(10,30))
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,:])
                axs[i,j].axis('off')
                cnt += 1
        fig.tight_layout()
        fig.savefig("images/%d.png" % epoch)
        plt.close()

    def save_model(self, epoch):

        def save(model, model_name, epoch):
            model_path = "saved_model/%s_%d.json" % (model_name, epoch)
            weights_path = "saved_model/%s_weights_%d.hdf5" % (model_name, epoch)
            options = {"file_arch": model_path,
                        "file_weight": weights_path}
            json_string = model.to_json()
            open(options['file_arch'], 'w').write(json_string)
            model.save_weights(options['file_weight'])

        save(self.generator, "generator", epoch)
        save(self.discriminator, "discriminator", epoch)


In [ ]:
if __name__ == '__main__':
    acgan = ACGAN()
    acgan.train(epochs=100000, batch_size=64, sample_interval=1000)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 64, 64, 16)   448         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (None, 64, 64, 16)   64          conv2d_9[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_9 (LeakyReLU)       (None, 64, 64, 16)   0           batch_normalization_9[0][0]      
__________________________________________________________________________________________________
lambda_4 (

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 2.091437, acc.: 42.19%, op_acc: 34.38%] [G loss: 1.814936]
1 [D loss: 1.861630, acc.: 54.69%, op_acc: 52.34%] [G loss: 1.972655]
2 [D loss: 1.608250, acc.: 64.06%, op_acc: 54.69%] [G loss: 2.069301]
3 [D loss: 1.668745, acc.: 66.41%, op_acc: 48.44%] [G loss: 2.210912]
4 [D loss: 1.614798, acc.: 61.72%, op_acc: 52.34%] [G loss: 2.234704]
5 [D loss: 1.500247, acc.: 75.00%, op_acc: 54.69%] [G loss: 2.579123]
6 [D loss: 1.418128, acc.: 73.44%, op_acc: 64.06%] [G loss: 2.544756]
7 [D loss: 1.379730, acc.: 80.47%, op_acc: 58.59%] [G loss: 2.560840]
8 [D loss: 1.374848, acc.: 78.91%, op_acc: 59.38%] [G loss: 2.423978]
9 [D loss: 1.566130, acc.: 57.03%, op_acc: 61.72%] [G loss: 2.048342]
10 [D loss: 1.361343, acc.: 80.47%, op_acc: 55.47%] [G loss: 2.225615]
11 [D loss: 1.380686, acc.: 82.03%, op_acc: 57.03%] [G loss: 2.208243]
12 [D loss: 1.279869, acc.: 80.47%, op_acc: 62.50%] [G loss: 1.928888]
13 [D loss: 1.590026, acc.: 65.62%, op_acc: 52.34%] [G loss: 1.798756]
14 [D loss: 1.30

118 [D loss: 0.585344, acc.: 100.00%, op_acc: 78.91%] [G loss: 0.517218]
119 [D loss: 0.543116, acc.: 100.00%, op_acc: 83.59%] [G loss: 0.488599]
120 [D loss: 0.355600, acc.: 100.00%, op_acc: 92.19%] [G loss: 0.522565]
121 [D loss: 0.537909, acc.: 100.00%, op_acc: 82.03%] [G loss: 0.432105]
122 [D loss: 0.499224, acc.: 99.22%, op_acc: 85.16%] [G loss: 0.461951]
123 [D loss: 0.449562, acc.: 100.00%, op_acc: 89.84%] [G loss: 0.562979]
124 [D loss: 0.592377, acc.: 98.44%, op_acc: 82.03%] [G loss: 0.496753]
125 [D loss: 0.512065, acc.: 100.00%, op_acc: 85.16%] [G loss: 0.659872]
126 [D loss: 0.481244, acc.: 99.22%, op_acc: 85.94%] [G loss: 0.460450]
127 [D loss: 0.423496, acc.: 100.00%, op_acc: 87.50%] [G loss: 0.510895]
128 [D loss: 0.493460, acc.: 98.44%, op_acc: 85.16%] [G loss: 0.844470]
129 [D loss: 0.576753, acc.: 95.31%, op_acc: 88.28%] [G loss: 1.439913]
130 [D loss: 0.619166, acc.: 96.09%, op_acc: 88.28%] [G loss: 0.537354]
131 [D loss: 0.470969, acc.: 100.00%, op_acc: 86.72%] [G 

232 [D loss: 0.442753, acc.: 99.22%, op_acc: 84.38%] [G loss: 0.118308]
233 [D loss: 0.315361, acc.: 100.00%, op_acc: 89.84%] [G loss: 0.090003]
234 [D loss: 0.343383, acc.: 100.00%, op_acc: 90.62%] [G loss: 0.199310]
235 [D loss: 0.373848, acc.: 100.00%, op_acc: 87.50%] [G loss: 0.072211]
236 [D loss: 0.294253, acc.: 100.00%, op_acc: 90.62%] [G loss: 0.094021]
237 [D loss: 0.267023, acc.: 100.00%, op_acc: 90.62%] [G loss: 0.091321]
238 [D loss: 0.359599, acc.: 100.00%, op_acc: 89.06%] [G loss: 0.102779]
239 [D loss: 0.362063, acc.: 100.00%, op_acc: 86.72%] [G loss: 0.069748]
240 [D loss: 0.378224, acc.: 100.00%, op_acc: 85.94%] [G loss: 0.104272]
241 [D loss: 0.364364, acc.: 99.22%, op_acc: 90.62%] [G loss: 0.217871]
242 [D loss: 0.282317, acc.: 100.00%, op_acc: 91.41%] [G loss: 0.112835]
243 [D loss: 0.255791, acc.: 98.44%, op_acc: 94.53%] [G loss: 0.086727]
244 [D loss: 0.337781, acc.: 100.00%, op_acc: 88.28%] [G loss: 0.129076]
245 [D loss: 0.254693, acc.: 100.00%, op_acc: 90.62%] 

...

5784 [D loss: 0.683003, acc.: 79.69%, op_acc: 96.09%] [G loss: 0.077975]
5785 [D loss: 0.125650, acc.: 97.66%, op_acc: 98.44%] [G loss: 0.078404]
5786 [D loss: 0.175869, acc.: 96.09%, op_acc: 98.44%] [G loss: 0.127174]
5787 [D loss: 0.205783, acc.: 96.88%, op_acc: 98.44%] [G loss: 0.178798]
5788 [D loss: 0.184898, acc.: 98.44%, op_acc: 98.44%] [G loss: 0.150968]
5789 [D loss: 0.132115, acc.: 98.44%, op_acc: 99.22%] [G loss: 0.037656]
5790 [D loss: 0.100223, acc.: 97.66%, op_acc: 99.22%] [G loss: 0.072187]
5791 [D loss: 0.516007, acc.: 83.59%, op_acc: 96.88%] [G loss: 0.075134]
5792 [D loss: 0.073627, acc.: 99.22%, op_acc: 99.22%] [G loss: 0.083357]
5793 [D loss: 0.066232, acc.: 99.22%, op_acc: 100.00%] [G loss: 0.278628]
5794 [D loss: 0.227291, acc.: 93.75%, op_acc: 99.22%] [G loss: 0.025349]
5795 [D loss: 0.119234, acc.: 98.44%, op_acc: 97.66%] [G loss: 0.097384]
5796 [D loss: 0.078327, acc.: 99.22%, op_acc: 99.22%] [G loss: 0.032486]
5797 [D loss: 0.365424, acc.: 90.62%, op_acc: 97.6

5896 [D loss: 0.416820, acc.: 84.38%, op_acc: 99.22%] [G loss: 0.032502]
5897 [D loss: 0.529212, acc.: 75.78%, op_acc: 98.44%] [G loss: 0.064695]
5898 [D loss: 0.075739, acc.: 98.44%, op_acc: 99.22%] [G loss: 0.139029]
5899 [D loss: 0.734413, acc.: 72.66%, op_acc: 97.66%] [G loss: 0.004506]
5900 [D loss: 0.145681, acc.: 94.53%, op_acc: 98.44%] [G loss: 0.112299]
5901 [D loss: 0.183723, acc.: 95.31%, op_acc: 99.22%] [G loss: 0.017630]
5902 [D loss: 0.144287, acc.: 97.66%, op_acc: 96.88%] [G loss: 0.019540]
5903 [D loss: 0.077218, acc.: 99.22%, op_acc: 98.44%] [G loss: 0.015470]
5904 [D loss: 0.212410, acc.: 98.44%, op_acc: 97.66%] [G loss: 0.079506]
5905 [D loss: 0.136843, acc.: 96.09%, op_acc: 99.22%] [G loss: 0.186625]
5906 [D loss: 0.039842, acc.: 100.00%, op_acc: 100.00%] [G loss: 0.255160]
5907 [D loss: 0.030750, acc.: 100.00%, op_acc: 99.22%] [G loss: 0.172865]
5908 [D loss: 0.111157, acc.: 97.66%, op_acc: 100.00%] [G loss: 0.103876]
5909 [D loss: 0.271909, acc.: 90.62%, op_acc: 1

6009 [D loss: 0.109696, acc.: 100.00%, op_acc: 96.88%] [G loss: 0.021074]
6010 [D loss: 0.065527, acc.: 100.00%, op_acc: 98.44%] [G loss: 0.010794]
6011 [D loss: 0.031749, acc.: 100.00%, op_acc: 99.22%] [G loss: 0.023532]
6012 [D loss: 0.099510, acc.: 100.00%, op_acc: 99.22%] [G loss: 0.591236]
6013 [D loss: 0.065125, acc.: 99.22%, op_acc: 99.22%] [G loss: 0.064640]
6014 [D loss: 0.252074, acc.: 94.53%, op_acc: 98.44%] [G loss: 0.028098]
6015 [D loss: 0.143491, acc.: 98.44%, op_acc: 99.22%] [G loss: 0.057351]
6016 [D loss: 0.273287, acc.: 86.72%, op_acc: 100.00%] [G loss: 0.002016]
6017 [D loss: 0.357740, acc.: 86.72%, op_acc: 99.22%] [G loss: 2.339411]
6018 [D loss: 0.516690, acc.: 88.28%, op_acc: 100.00%] [G loss: 0.772765]
6019 [D loss: 0.146568, acc.: 96.09%, op_acc: 99.22%] [G loss: 0.002783]
6020 [D loss: 0.048818, acc.: 99.22%, op_acc: 100.00%] [G loss: 0.006149]
6021 [D loss: 0.052299, acc.: 100.00%, op_acc: 99.22%] [G loss: 0.004967]
6022 [D loss: 0.091667, acc.: 99.22%, op_ac

6121 [D loss: 0.037127, acc.: 100.00%, op_acc: 100.00%] [G loss: 0.001561]
6122 [D loss: 0.070806, acc.: 99.22%, op_acc: 96.88%] [G loss: 0.077534]
6123 [D loss: 0.126434, acc.: 97.66%, op_acc: 100.00%] [G loss: 0.001260]
6124 [D loss: 0.078258, acc.: 99.22%, op_acc: 99.22%] [G loss: 0.012195]
6125 [D loss: 0.033009, acc.: 100.00%, op_acc: 100.00%] [G loss: 0.018374]
6126 [D loss: 0.072630, acc.: 98.44%, op_acc: 99.22%] [G loss: 0.007946]
6127 [D loss: 0.107622, acc.: 97.66%, op_acc: 99.22%] [G loss: 0.003320]
6128 [D loss: 0.028521, acc.: 99.22%, op_acc: 100.00%] [G loss: 0.023522]
6129 [D loss: 0.061082, acc.: 100.00%, op_acc: 100.00%] [G loss: 0.015041]
6130 [D loss: 0.022166, acc.: 100.00%, op_acc: 100.00%] [G loss: 0.039879]
6131 [D loss: 0.042821, acc.: 99.22%, op_acc: 100.00%] [G loss: 0.543353]
6132 [D loss: 0.210675, acc.: 96.88%, op_acc: 96.88%] [G loss: 0.013831]
6133 [D loss: 0.178671, acc.: 95.31%, op_acc: 100.00%] [G loss: 0.009873]
6134 [D loss: 0.040605, acc.: 100.00%, 

6233 [D loss: 0.047828, acc.: 100.00%, op_acc: 98.44%] [G loss: 0.060828]
6234 [D loss: 0.096930, acc.: 100.00%, op_acc: 96.88%] [G loss: 0.072081]
6235 [D loss: 0.074033, acc.: 99.22%, op_acc: 98.44%] [G loss: 0.008192]
6236 [D loss: 0.035154, acc.: 100.00%, op_acc: 98.44%] [G loss: 0.006938]
6237 [D loss: 0.040358, acc.: 100.00%, op_acc: 98.44%] [G loss: 0.011462]
6238 [D loss: 0.055491, acc.: 100.00%, op_acc: 98.44%] [G loss: 0.010845]
6239 [D loss: 0.021006, acc.: 100.00%, op_acc: 100.00%] [G loss: 0.028324]
6240 [D loss: 0.049173, acc.: 100.00%, op_acc: 99.22%] [G loss: 0.004751]
6241 [D loss: 0.020137, acc.: 100.00%, op_acc: 99.22%] [G loss: 0.012802]
6242 [D loss: 0.035206, acc.: 100.00%, op_acc: 100.00%] [G loss: 0.155049]
6243 [D loss: 0.045488, acc.: 99.22%, op_acc: 99.22%] [G loss: 0.002990]
6244 [D loss: 0.064389, acc.: 100.00%, op_acc: 97.66%] [G loss: 0.004983]
6245 [D loss: 0.066173, acc.: 100.00%, op_acc: 97.66%] [G loss: 0.006317]
6246 [D loss: 0.034091, acc.: 100.00%,